# Network



In [0]:
!pip install pydicom

import os
import csv
import random
import pydicom
import numpy as np
import pandas as pd
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt

In [0]:


cwd = os.getcwd() 
print(cwd)

os.chdir('/content/drive/My Drive/LUNG/')

cwd = os.getcwd() 
print(cwd)

/content
/content/drive/My Drive/LUNG


# Load ground truth locations


In [0]:
# empty dictionary
nodule_locations = {}
# load table
with open(os.path.join('./stage_1_train_labels.csv'), mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader, None)
    # loop through rows
    for rows in reader:
        # retrieve information
        filename = rows[0]
        location = rows[1:5]
        sick = rows[5]
        # if row contains a nodule add label to dictionary
        # which contains a list of nodule locations per filename
        if sick == '1':
            # convert string to float to int
            location = [int(float(i)) for i in location]
            # save nodule location in dictionary
            if filename in nodule_locations:
                nodule_locations[filename].append(location)
            else:
                nodule_locations[filename] = [location]

# Load filenames

In [0]:
# load and shuffle filenames
folder = './stage_1_train_images'
filenames = os.listdir(folder)
random.seed(42)
random.shuffle(filenames)
# split into train and validation filenames
n_valid_samples = 1500
train_filenames = filenames[n_valid_samples:]
valid_filenames = filenames[:n_valid_samples]
print('n train samples', len(train_filenames))
print('n valid samples', len(valid_filenames))
n_train_samples = len(filenames) - n_valid_samples

('n train samples', 24214)
('n valid samples', 1500)


 # Data generator

The dataset is too large to fit into memory, so a generator is made that loads data on the fly.

* The generator takes in some filenames, batch_size and other parameters.

* The generator outputs a random batch of numpy images and numpy masks.
    

In [0]:
class generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, nodule_locations=None, batch_size=32, image_size=512, shuffle=True, predict=False, augment = False):
        self.folder = folder
        self.filenames = filenames
        self.nodule_locations = nodule_locations
        self.batch_size = batch_size
        self.image_size = image_size
        self.augment = augment
        self.shuffle = shuffle
        self.predict = predict
        self.on_epoch_end()
        
    def __load__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # create empty mask
        msk = np.zeros(img.shape)
        # get filename without extension
        filename = filename.split('.')[0]
        # if image contains nodules
        if filename in nodule_locations:
            # loop through nodules
            for location in nodule_locations[filename]:
                # add 1's at the location of the nodule
                x, y, w, h = location
                msk[y:y+h, x:x+w] = 1
        # resize both image and mask
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        msk = resize(msk, (self.image_size, self.image_size), mode='reflect') > 0.5
        # if augment then horizontal flip half the time
        if self.augment and random.random() > 0.5:
            img = np.fliplr(img)
            msk = np.fliplr(msk)
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)
        return img, msk
    
    def __loadpredict__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # resize image
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img
        
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # predict mode: return images and filenames
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, filenames
        # train mode: return images and masks
        else:
            # load files
            items = [self.__load__(filename) for filename in filenames]
            # unzip images and masks
            imgs, msks = zip(*items)
            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
        
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            # return full batches only
            return int(len(self.filenames) / self.batch_size)

# Network


In [0]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization()(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization()(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=5):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
    # output
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Train network


In [0]:
# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

# create network and compiler
model = create_network(input_size=256, channels=32, n_blocks=2, depth=4)
model.compile(optimizer=keras.optimizers.Adam(lr=.0001),loss=keras.losses.binary_crossentropy,metrics=['accuracy', mean_iou])

# weights
model_weight = './weights-improvement-22-0.98.h5'
from keras.callbacks import ModelCheckpoint

# Save the model
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# create train and validation generators
folder = './stage_1_train_images'
train_gen = generator(folder, train_filenames, nodule_locations, batch_size=32, image_size=256, shuffle=True, augment=True, predict=False)
valid_gen = generator(folder, valid_filenames, nodule_locations, batch_size=32, image_size=256, shuffle=False, predict=False)

model.load_weights(model_weight)
history = model.fit_generator(train_gen, validation_data=valid_gen, epochs=50, callbacks=callbacks_list, shuffle=True, verbose=1, initial_epoch=22)


plt.figure(figsize=(12,4))
plt.subplot(131)
plt.plot(history.epoch, history.history["loss"], label="Train loss")
plt.plot(history.epoch, history.history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(132)
plt.plot(history.epoch, history.history["acc"], label="Train accuracy")
plt.plot(history.epoch, history.history["val_acc"], label="Valid accuracy")
plt.legend()
plt.subplot(133)
plt.plot(history.epoch, history.history["mean_iou"], label="Train iou")
plt.plot(history.epoch, history.history["val_mean_iou"], label="Valid iou")
plt.legend()
plt.savefig('loss.png')
plt.show()

Using TensorFlow backend.


# Predict test images

In [0]:
# empty dictionary
nodule_locations_new = {}
# load table
with open(os.path.join('./stage_1_train_labels.csv'), mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader, None)
    # loop through rows
    for rows in reader:
        # retrieve information
        filename = rows[0]
        location = rows[1:5]
        nodule = rows[5]
        # if row contains a nodule add label to dictionary
        # which contains a list of nodule locations per filename
        if nodule == '1':
            # convert string to float to int
            location = [int(float(i)) for i in location]
            # save nodule location in dictionary
            if filename in nodule_locations_new:
                nodule_locations_new[filename].append(location)
            else:
                nodule_locations_new[filename] = [location]
        else:
            location = [0,0,0,0]
            nodule_locations_new[filename] = [location]



# load and shuffle filenames
folder = './stage_1_train_images'

val_gen = generator(folder, valid_filenames, None, batch_size=32, image_size=256, shuffle=True, predict=True)

# loop through valset
c = 0
stroke=0
rgb = [255, 251, 204] 


fig = plt.figure(figsize=(100, 300))
for imgs, filename in val_gen:
    if c in range(10):
      
      # predict batch of images and resize
      pred = model.predict(imgs)
      truestring= filename[0]
      filename = str(truestring[:-4])
      pred = pred[0,:,:,:]
      imgs = imgs[0,:,:,:]
      pred = resize(pred, (1024, 1024), mode='reflect')
      imgs = resize(imgs, (1024, 1024), mode='reflect')
      imgs = imgs[:,:,0]

      # threshold predicted mask
      comp1 = pred[:, :, 0] > 0.5
      # apply connected components
      comp2 = measure.label(comp1)

      
      
      # Heat map      
      plt.subplot(10, 2, 2*c + 1)
      imgstru = imgs
      imgstru = np.stack([imgstru] * 3, axis=2)
      plt.imshow(imgstru, cmap=plt.cm.gist_gray)

      for location in nodule_locations_new[filename]:
            x1, y1, width, height = location 
            if x1!=0 and y1!=0 and width!=0 and height!=0:
  
              y1 = int(y1)
              x1 = int(x1)

              rectangle_gap = plt.Rectangle((x1,y1),width,height,linewidth=10, linestyle = '--', edgecolor='r',fill = False, facecolor='none')
              plt.gca().add_patch(rectangle_gap)

              continue
 
      plt.axis('off') 
      con = []
      plt.subplot(10, 2, 2*c + 2)
      imgsval = imgs
      imgsval = np.stack([imgsval] * 3, axis=2)
      plt.imshow(imgsval, cmap=plt.cm.gist_gray)

      for region in measure.regionprops(comp2):

            # retrieve x, y, height and width
            y1, x1, y2, x2 = region.bbox
            h = int(y2 - y1)
            w = int(x2 - x1)
            y1 = int(y1)
            x1 = int(x1)

            rectangle_gap = plt.Rectangle((x1,y1),w,h,linewidth=10,linestyle = '--', edgecolor='r',fill = False,facecolor='none')
            plt.gca().add_patch(rectangle_gap)              

      plt.axis('off') 
    else:
      break
    c+=1
    
    
plt.tight_layout()    
plt.show()    
fig.savefig('truth vs. predictions2.jpg')

In [0]:
# load and shuffle filenames
folder = './stage_1_test_images'
test_filenames = os.listdir(folder)
print('n test samples:', len(test_filenames))

# create test generator with predict flag set to True
test_gen = generator(folder, test_filenames, None, batch_size=25, image_size=256, shuffle=False, predict=True)

# create submission dictionary
submission_dict = {}
# loop through testset
for imgs, filenames in test_gen:
    # predict batch of images
    preds = model.predict(imgs)
    #print(preds.shape)
    # loop through batch
    for pred, filename in zip(preds, filenames):
        #print(pred.shape)

        # resize predicted mask
        pred = resize(pred, (1024, 1024), mode='reflect')
        
        #print(pred.shape)

        # threshold predicted mask
        comp = pred[:, :, 0] > 0.7
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x

            # proxy for confidence score
            conf = np.mean(pred[y:y+height, x:x+width])
            # add to predictionString
            predictionString += str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) + ' '
        # add filename and predictionString to dictionary
        filename = filename.split('.')[0]
        submission_dict[filename] = predictionString
    # stop if we've got them all
    if len(submission_dict) >= len(test_filenames):
        break

# save dictionary as csv file
sub = pd.DataFrame.from_dict(submission_dict,orient='index')
sub.index.names = ['patientId']
sub.columns = ['PredictionString']
sub.to_csv('submission1_conf=0.7.csv')